Este codigo aplica la primera parte del ETL. En particular se realiza lo siguiente:
- Desanidar valores en columnas
- Relleno de valores nulos
- Crear columna release_year
- Crear columna return
- Eliminar columnas que no se utilizan 

Al finalizar, se almacena el dataset nuevo como 'movies_etl.csv'

Por lo demas, el codigo esta debidamente documentado con comentarios que explican lo que se realizo en cada caso.

In [1]:
import pandas as pd
import ast # esto es para usar ast.literal_eval sobre str en formato dict/list/int/float
import numpy as np

In [2]:
path = '../data/movies_dataset.csv'
original = pd.read_csv(path, low_memory=False)
modify = original.copy()

In [3]:
modify.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
modify.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

**Transformaciones**:

1. Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

2. Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

3. De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

4. Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

5. Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,vote_count,poster_path y homepage.



In [5]:
modify.shape

(45466, 24)

Columnas con valores anidados

belongs_to_collection
genres
production_companies
production_countries
spoken_languages

In [6]:
dict_columns = ['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages']

In [7]:
# ejemplo de formato

for c in dict_columns:
    print(modify.loc[4, c])
    
# hay dos formas: o bien es una lista de diccionarios, o bien es un diccionario. 
# Estos dos tipos lo contempla la funcion de abajo

{'id': 96871, 'name': 'Father of the Bride Collection', 'poster_path': '/nts4iOmNnq7GNicycMJ9pSAn204.jpg', 'backdrop_path': '/7qwE57OVZmMJChBpLEbJEmzUydk.jpg'}
[{'id': 35, 'name': 'Comedy'}]
[{'name': 'Sandollar Productions', 'id': 5842}, {'name': 'Touchstone Pictures', 'id': 9195}]
[{'iso_3166_1': 'US', 'name': 'United States of America'}]
[{'iso_639_1': 'en', 'name': 'English'}]


In [8]:
def extract_name(value):
    ''' extrae de los valores anidados, el valor del campo "name" ''' 
    output = []
    try:
        value = ast.literal_eval(value)
        if isinstance(value, list):
            for dictionary in value:
                output.append(dictionary['name'])
        elif isinstance(value, dict):
            output.append(value['name'])
    finally:
        for i in range(len(output)):
            output[i] = output[i].lower()
        return output
    
def extract_iso(value):
    ''' extrae de los valores anidados, el valor del campo "iso" ''' 
    output = []
    try:
        value = ast.literal_eval(value)
        if isinstance(value, list):
            for dictionary in value:
                for key, value in dictionary.items():
                    if 'iso' in key:
                        output.append(value)
    finally:
        for i in range(len(output)):
            output[i] = output[i].lower()
        return output
    
def is_float(column):
    ''' chequea si existe algun valor en column que tenga decimales ''' 
    for v in column:
        try:
            test = v - int(v)
            if test > 0:
                return True
        except:
            continue
    return False

def type_data(value):
    ''' convierte datos de tipo str a float/int '''
    try:
        return ast.literal_eval(value)
    except:
        print(value)
        return np.nan

In [9]:
# extraemos los valores anidados y creamos nuevas columnas con el sufijo "_mod"
for c in dict_columns:
    modify[c + '_mod'] = modify[c].apply(extract_name)

In [10]:
# extraemos los valores anidados y creamos nuevas columnas con el sufijo "_iso"
for c in dict_columns[-2:]:
    modify[c + '_iso'] = modify[c].apply(extract_iso)

In [11]:
# chequeamos si tiene valores flotantes o no las siguientes columnas
print(is_float(modify.budget))
print(is_float(modify.revenue))
print(is_float(modify.popularity))

False
False
False


In [12]:
# columna budget

# chequeamos el numero de valores faltantes
print('antes de la transf:', modify.budget.isnull().sum())

# convertimos el str a valor numerico usando type_data()
modify.budget = modify.budget.apply(type_data)

# llenamos los valores faltantes con 0 y convertimos la columna a "int"
modify.budget = modify.budget.fillna(0).astype('int')

# confirmamos la operacion
print('despues de la transf:', modify.budget.isnull().sum())

antes de la transf: 0
/ff9qCepilowshEtG2GYWwzt2bs4.jpg
/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg
/zaSf5OG7V8X8gqFvly88zDdRm46.jpg
despues de la transf: 0


In [13]:
# columna revenue

# llenamos los valores faltantes con 0 y convertimos la columna a "int"
modify.revenue = modify.revenue.fillna(0).astype('int')

# convertimos los valores negativos a positivo (es 1 solo)
modify.revenue = modify.revenue.apply(lambda v: v if v > 0 else v * (-1))

In [14]:
# columna popularity

# convertimos los valores str a float.
modify.popularity = modify.popularity.apply(type_data)

nan
nan
nan
nan
nan
Beware Of Frost Bites


In [15]:
def date_data(value):
    ''' 
    chequea si el valor es de tipo date:
        si es, lo devuelve value
        si no es, devuelve NA
    '''
    try:
        year, month, day = value.split('-')
        return value
    except:
        return np.nan

In [16]:
# columna release_date

# ejemplo de formato fecha

print('ejemplo fecha:', modify.release_date[0])

# numero de nulos para release_date sin modificacion.
print(modify.release_date.isnull().sum())

# convierte a NA todos los valores sin el formato fecha.
modify.release_date = modify.release_date.apply(date_data)

# convierte la columna a datetime
modify.release_date = pd.to_datetime(modify.release_date)

# numero de nulos nuevos
print(modify.release_date.isnull().sum())

# eliminamos las filas con nulos
modify = modify.dropna(axis=0, subset=['release_date'])

ejemplo fecha: 1995-10-30
87
90


In [17]:
# creamos una columna llamada release_year, que contiene el anio de estreno
modify['release_year'] = modify.release_date.dt.year

In [18]:
# creamos la columna return: porcentaje de ganancia respecto de lo invertido.

modify['return'] = modify.revenue / modify.budget
modify['return'] = modify['return'].replace([np.inf, -np.inf], np.nan)
modify['return'] = modify['return'].fillna(0)

In [19]:
# columna id

modify.id = modify.id.apply(type_data)

In [20]:
# observamos las columnas que tenemos, asi decidimos cual eliminamos y con cual nos quedamos

modify.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'belongs_to_collection_mod', 'genres_mod',
       'production_companies_mod', 'production_countries_mod',
       'spoken_languages_mod', 'production_countries_iso',
       'spoken_languages_iso', 'release_year', 'return'],
      dtype='object')

In [21]:
# eliminamos las columnas que no vamos a utilizar

columnas=['video', 'imdb_id', 'adult', 'original_title', 'vote_count', 'poster_path', 'homepage',
          'belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages']

# creamos un nuevo dataframe, que es el que vamos a almacenar.
output = modify.drop(columns=columnas)

In [22]:
# convertimos las columnas a su dtype adecuado

# no lo hacemos para overview ya que vamos a aplicar nlp, y las mayusculas pueden ayudar al modelo a entender el contexto.
str_columns = ['original_language', 'status', 'tagline', 'title'] # overview
int_columns = ['budget', 'id', 'revenue', 'runtime', 'release_year']
float_columns = ['popularity', 'vote_average', 'return']

for c in str_columns:
    output[c] = output[c].str.lower()
    
# for c in int_columns:
#     output[c] = output[c].astype('int32', errors='ignore')
    
for c in float_columns:
    output[c] = output[c].astype('float64')

In [23]:
# modificamos el nombre de algunas columnas

output = output.rename(columns=dict(
            belongs_to_collection_mod='belongs_to_collection',
            genres_mod='genres',
            production_companies_mod='production_companies',
            production_countries_mod='production_countries',
            spoken_languages_mod='spoken_languages'))

In [24]:
# creamos las columnas con los meses y dias del release.

meses = {'october': 'octubre', 'december': 'diciembre', 'february': 'febrero', 'november': 'noviembre', 'september': 'septiembre',
     'may': 'mayo', 'april': 'abril', 'august': 'agosto', 'july': 'julio', 'june': 'junio', 'january': 'enero','march': 'marzo'}

dias = {'monday': 'lunes','friday': 'viernes', 'thursday': 'martes', 'wednesday': 'miercoles',
     'saturday': 'sabado', 'tuesday': 'jueves', 'sunday': 'domingo'}

output['release_month'] = output.release_date.dt.month_name().str.lower().replace(meses)
output['release_day'] = output.release_date.dt.day_name().str.lower().replace(dias)

In [25]:
# eliminamos duplicados: consideramos duplicado aquella misma pelicula con un mismo release_year

output = output.drop_duplicates(subset=['title', 'release_year'], ignore_index=True)

In [26]:
# lo almacenamos en un formato csv el nuevo dataset

output.to_csv('../data/movies_etl.csv', sep=',', index=False, header=True)

In [27]:
# base_poster_url = 'http://image.tmdb.org/t/p/w185/'
# modify['image_path'] = "<img src='" + base_poster_url + modify['poster_path'] + "' style='height:100px;'>"